In [1]:
def simplex(matrix, rhs, z, varlabel='x',  direction=1):
    '''Simplex algorithm to solve linear programming problems

    Parameters
    ----------
    matrix: numpy ndarray
        Matrix of coefficients in the left-hand side

    rhs: numpy ndarray
        Right-hand side vector


    direction: {+1 , -1}
        Use +1 for maximization and -1 for minimization.

    Returns
    -------
    optimal_table: Pandas DataFrame
        Optimal Table
    
    b: pandas Series
        Vector solution
    
    cbindex: Pandas Series
        Index of coefficient basics in z vector
    '''
    print("="*10, "Simplex Method", "="*10)

    matrix = np.array(matrix, dtype=float)
    num_rows, num_cols = matrix.shape
    
    rhs = np.array(rhs, dtype=float)
    z = np.array(z, dtype=float)
    
    
    cb_index = np.where((matrix == 1) & (matrix.sum(axis=0) == 1))[1]
    cb = z[cb_index]
    zj = cb.dot(matrix)
    net_evaluation = direction * (z - zj)
    solutions = []
    

    labels = [f"{varlabel}{i + 1}" for i in range(num_cols)]
    basis = [labels[i] for i in cb_index]
    
    while np.any(net_evaluation > 0):
        # ======== Feasible Condition ==========
        entering = net_evaluation.argmax()  # entering variables (index)
        entering_label = labels[entering]

        key_col = matrix[:, entering]

        ratios = np.divide(rhs, key_col,
                           out=np.full_like(rhs, np.inf),
                           where=key_col > 0)

        leaving = ratios.argmin()   # leaving variables (index)
        leaving_label = labels[cb_index[leaving]]
        # ======= Gauss-Jordan =============
        pivot = matrix[leaving, entering]

        if pivot != 1:
            matrix[leaving] = matrix[leaving] / pivot
            rhs[leaving] = rhs[leaving] / pivot

        
        for i in range(num_rows):
            if i != leaving:
                factor = matrix[i, entering]
                matrix[i] = -factor * matrix[leaving] + matrix[i]
                rhs[i] = -factor * rhs[leaving] + rhs[i]
        # ======== Update ==========
        cb_index[leaving] = entering

        cb = z[cb_index]
        zj = cb.dot(matrix)
        net_evaluation = direction * (z - zj)

        
        basis[leaving] = entering_label  

        solutions.append(rhs)
        # ======== Optimality Condition ==========
        if np.all(net_evaluation <= 0):
            solution = np.zeros_like(z)
            solution[cb_index] = rhs
            print(f"Optimal solution found in {len(solutions)} iterations")
            print(f'Optimal Table: \n {matrix} \n')
            print("Solution", solution, f"\tZ: {cb.dot(rhs):0.2f}")
            print("="*10, "Simplex Method", "="*10)
            
            data_solution = np.hstack((rhs[:,np.newaxis], z[cb_index][:, np.newaxis], cb_index[:, np.newaxis]))            
            data_df = pd.DataFrame(data_solution, index=basis, columns=['b', 'cb', 'cbidx'])
            matrix_df = pd.DataFrame(matrix, index=basis, columns=labels)
            
            return pd.concat([matrix_df, data_df], axis=1)        

In [ ]:
def last_rows(cj, cb,  matrix, label='x'):
    assert cj.size == matrix.shape[1], 'Lenghts are different'
    zj = cb.dot(matrix)
    net_profit = cj - zj
    lastrows = np.vstack((zj, net_profit))
    labels = [f'x{i + 1}' for i in range(cj.size)]
    return pd.DataFrame(lastrows, index=['zj', 'cj - zj'], columns=labels)